In [53]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Open Housing data

In [54]:
homeless_change_df = pd.read_excel(
    "data/2007-2023-PIT-Counts-by-State.xlsb", engine="pyxlsb"
)

In [55]:
homeless_change_df = homeless_change_df.rename(columns={"State": "STUSPS"})

## Open GIS Data

In [56]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Population Data

In [57]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [58]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [68]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Merge Data for Homeless Changes

In [69]:
homeless_per_state_changes_df = states_with_population_df.merge(
    homeless_change_df, on="STUSPS", how="left"
)

In [70]:
homeless_per_state_changes_df = homeless_per_state_changes_df.to_crs(5070)

In [71]:
change_cols = [
    "Change in Total Homelessness, 2022-2023",
    "Change in Total Homelessness, 2021-2023",
    "Change in Total Homelessness, 2020-2023",
    "Change in Total Homelessness, 2019-2023",
    "Change in Total Homelessness, 2018-2023",
    "Change in Total Homelessness, 2017-2023",
    "Change in Total Homelessness, 2016-2023",
    "Change in Total Homelessness, 2015-2023",
    "Change in Total Homelessness, 2014-2023",
    "Change in Total Homelessness, 2013-2023",
    "Change in Total Homelessness, 2012-2023",
    "Change in Total Homelessness, 2011-2023",
    "Change in Total Homelessness, 2010-2023",
    "Change in Total Homelessness, 2009-2023",
    "Change in Total Homelessness, 2008-2023",
    "Change in Total Homelessness, 2007-2023",
]

In [72]:
homeless_per_state_changes_df = homeless_per_state_changes_df[
    ["STUSPS", "geometry", *change_cols]
]

In [64]:
homeless_per_state_changes_df.replace(" ", np.nan, inplace=True)
homeless_per_state_changes_df.dropna(inplace=True)

/tmp/ipykernel_14124/3328568637.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  homeless_per_state_changes_df.replace(' ', np.nan, inplace=True)


In [66]:
for col in change_cols:
    homeless_per_state_changes_df[col] = homeless_per_state_changes_df[col].astype(
        float
    )

In [67]:
homeless_per_state_changes_df.to_file("data/homeless_changes.gpkg")